In [1]:
# https://towardsdatascience.com/text-summarization-using-deep-neural-networks-e7ee7521d804/  
https://saadsohail5104.medium.com/understanding-padding-in-nlp-types-and-when-to-use-them-bacae6cae401
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/data_util/data.py
https://www.ijcaonline.org/archives/volume185/number15/patil-2023-ijca-922837.pdf
https://www.kaggle.com/code/getanmolgupta01/text-summarization-lstm-encoder-decoder
https://github.com/rohithreddy024/Text-Summarizer-Pytorch/blob/master/model.py
https://arxiv.org/abs/1508.04025

SyntaxError: invalid decimal literal (2005343661.py, line 4)